<a href="https://colab.research.google.com/github/Mierinn/Decentrathon-4.0/blob/main/decentrathon_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from folium.plugins import HeatMap
from google.colab import drive
from IPython.display import display


drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/geo_locations_astana_hackathon/geo_locations_astana_hackathon')


df_sample = df.sample(n=50000, random_state=1)
points = df_sample[['lat', 'lng']].values.tolist()

center_lat = df_sample['lat'].mean()
center_lng = df_sample['lng'].mean()


m = folium.Map(location=[center_lat, center_lng], zoom_start=12)
HeatMap(points, radius=8, blur=4).add_to(m)


kmeans = KMeans(n_clusters=3, random_state=1, n_init=10)
labels = kmeans.fit_predict(df_sample[['lat', 'lng']])
centers = kmeans.cluster_centers_


best_points = []
for i, center in enumerate(centers):
    cluster_points = df_sample[labels == i][['lat', 'lng']].values
    distances = np.linalg.norm(cluster_points - center, axis=1)
    closest_point = cluster_points[np.argmin(distances)]
    best_points.append(closest_point)


types = ["Дорожные работы", "Пробка", "Авария"]


marker_styles = {
    "Дорожные работы": ("orange", "wrench"),
    "Пробка": ("red", "car"),
    "Авария": ("darkred", "exclamation-triangle")
}


for i, (lat, lng) in enumerate(best_points):
    t = types[i]
    color, icon = marker_styles[t]
    folium.Marker(
        [lat, lng],
        popup=f"{t} (кластер {i+1})",
        icon=folium.Icon(color=color, icon=icon, prefix="fa")
    ).add_to(m)


display(m)

m.save("/content/drive/MyDrive/geo_locations_astana_hackathon/map_with_markers.html")
print("Карта сохранена в Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Карта сохранена в Google Drive.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from folium.plugins import HeatMap
from google.colab import drive
from IPython.display import display


drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/geo_locations_astana_hackathon/geo_locations_astana_hackathon')


df_sample = df.sample(n=50000, random_state=1)
points = df_sample[['lat', 'lng']].values.tolist()

center_lat = df_sample['lat'].mean()
center_lng = df_sample['lng'].mean()


m = folium.Map(location=[center_lat, center_lng], zoom_start=12)
HeatMap(points, radius=8, blur=4).add_to(m)

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init=10)
labels = kmeans.fit_predict(df_sample[['lat', 'lng']])
centers = kmeans.cluster_centers_


recommendations = []
for i, center in enumerate(centers):
    cluster_points = df_sample[labels == i][['lat', 'lng']].values
    demand = len(cluster_points)

    drivers_needed = max(1, demand // 200)

    recommendations.append({
        "cluster": i+1,
        "lat": center[0],
        "lng": center[1],
        "demand": demand,
        "drivers": drivers_needed
    })


for rec in recommendations:
    folium.CircleMarker(
        [rec["lat"], rec["lng"]],
        radius=10 + rec["drivers"],
        color="blue",
        fill=True,
        fill_opacity=0.6,
        popup=f"Кластер {rec['cluster']}<br>Спрос: {rec['demand']}<br>Водителей нужно: {rec['drivers']}"
    ).add_to(m)


display(m)


m.save("/content/drive/MyDrive/geo_locations_astana_hackathon/optimized_drivers.html")
print("Карта сохранена в Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Карта сохранена в Google Drive.
